Using Capitals.txt dataset;

Find two closest capital cities and the distance between them.

In [23]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark


Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:11 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2,277 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,111 kB]
Get:14 http://archive.ubun

In [24]:
from pyspark import SparkContext
# import findspark
# findspark.init()

# import file
sc = SparkContext.getOrCreate()
capitalRDD = sc.textFile("Capitals.txt")

# Split data, remove spaces, arrange to make float, make float, replace longitude and latitude if they are disorder.
def split_data(line):
    info = line.split("\t")
    country = info[1].strip()
    capital = country + ", " + info[2].strip().replace("'", " ")
    latitude = float(info[3].strip().replace(",", "."))
    longitude = float(info[4].strip().replace(",", "."))
    if latitude > 90 or latitude < -90:
        return (capital, (longitude, latitude))
    else:
        return (capital, (latitude, longitude))


capitalRDD = capitalRDD.map(lambda line: split_data(line))
capitalRDD.take(10)


[('Afghanistan, Kabul', (34.53, 69.17)),
 ('Albania, Tirana', (41.33, 19.82)),
 ('Algeria, Algiers', (36.75, 3.04)),
 ('American Samoa, Pago Pago', (-14.28, -170.7)),
 ('Andorra, Andorra la Vella', (42.51, 1.52)),
 ('Angola, Luanda', (-8.84, 13.23)),
 ('Anguilla, The Valley', (18.22, -63.06)),
 ('Antigua and Barbuda, St. John s', (17.12, -61.85)),
 ('Argentina, Buenos Aires', (-34.61, -58.38)),
 ('Armenia, Yerevan', (40.18, 44.51))]

In [25]:
import geopy.distance

def calculate_distance(coords1, coords2):
    distance = geopy.distance.geodesic(coords1, coords2).km
    distance = round(distance,2)
    return distance

# create combinations using cartesian function
combinationRDD = capitalRDD.cartesian(capitalRDD)
# calculate distance for each combination
distancesRDD = combinationRDD.map(lambda x: (x[0][0], x[1][0], calculate_distance(x[0][1], x[1][1])))
# Sort and filter distances
distancesRDD = distancesRDD.filter(lambda x: x[2] != 0)
distancesRDD = distancesRDD.sortBy(lambda x :x[2])

# get closest capitals
closest_combination = distancesRDD.take(1)
closest_combination
print("Closest capital cities in the world: ", closest_combination[0][0], " and ", closest_combination[0][1], ". Distance (km):", closest_combination[0][2])


Closest capital cities in the world:  Italy, Rome  and  Vatican, Vatican . Distance (km): 2.73
